# CREATING FACT TABLE

### Reading data from silver layer

In [0]:
%sql
SELECT * FROM PARQUET.`abfss://silver@carsneelabhdatalake.dfs.core.windows.net/carsales`

# FETCHING RELEVANT COLUMNS

In [0]:
df_silver = spark.sql("SELECT * FROM PARQUET.`abfss://silver@carsneelabhdatalake.dfs.core.windows.net/carsales`")

In [0]:
df_silver.display()

### READING ALL THE DIMENSIONS

In [0]:
df_dealer = spark.sql("select * from cars_catalog.gold.dim_dealer")

df_model = spark.sql("select * from cars_catalog.gold.dim_model")

df_branch = spark.sql("select * from cars_catalog.gold.dim_branch")

df_date = spark.sql("select * from cars_catalog.gold.dim_date")

## Bringing foreign keys to the Fact tables

In [0]:
df_fact = df_silver.join(df_branch,df_silver['Branch_ID']== df_branch['BRANCH_ID'],how='left')\
            .join(df_dealer,df_silver['Dealer_ID']== df_dealer['DEALER_ID'],how='left')\
            .join(df_model,df_silver['Model_ID']== df_model['MODEL_ID'],how='left')\
            .join(df_date,df_silver['Date_ID']== df_date['DATE_ID'],how='left')\
            .select(df_silver['Revenue'],df_silver['Units_Sold'],df_silver['RevPerUnit'],df_branch['dim_branch_key'],df_dealer['dim_dealer_key'],df_model['dim_model_key'],df_date['dim_date_key']) 


In [0]:
df_fact.display()

# Writing Fact table

In [0]:
from delta.tables import DeltaTable

In [0]:
if spark.catalog.tableExists("fact_sales"):
    deltatbl = DeltaTable.forName(spark, "cars_catalog.gold.fact_sales")
    deltatbl.alias("trg").merge(df_fact.alias('src'),'trg.dim_branch_key = src.dim_branch_key and trg.dim_dealer_key = src.dim_dealer_key and trg.dim_model_key = src.dim_model_key and trg.dim_date_key = src.dim_date_key')\
        .whenMatchedUpdateAll()\
            .whenNotMatchedInsertAll()\
                .execute()
       

else:
    df_fact.write.format('delta')\
        .mode('Overwrite')\
            .option("path", "abfss://bronze@carsneelabhdatalake.dfs.core.windows.net/fact_sales")\
                .saveAsTable("cars_catalog.gold.fact_sales")

In [0]:
%sql
select * from cars_catalog.gold.fact_sales